# 02h – 9. Experiment: LLM-Postprocessing v3: Regelbasierter Ansatz mit Qwen2.5-Coder

## Motivation

`02f`/`02g` nutzten ein generisches SEP-basiertes Block-Prompting-Schema.
Dieses Notebook testet einen grundlegend anderen Ansatz:

**Idee:** Domänen-spezifische ASR-Fehler (Eigennamen, Filmtitel, häufige Verwechslungen)
sind bekannt. Ein LLM das explizite Korrekturregeln als Prompt erhält und
Zeile für Zeile arbeitet, könnte gezielter korrigieren.

| Merkmal | `02f`/`02g` | `02h` (dieser Ansatz) |
|---------|-------------|----------------------|
| Prompting | SEP-Block-Schema | Flexibler Freitext-Prompt |
| Regeln | Keine expliziten | CORRECTION_RULES-Dict |
| Decoding | Greedy (do_sample=False) | Sampling (temperature=0.3) |
| Granularität | 4–8 Zeilen/Aufruf | 1 Zeile/Aufruf |
| Modell | Qwen3-8B (bestes aus 02f) | Qwen2.5-Coder-7B-Instruct |

## Ergebnis (Vorschau)

E43 verschlechtert die Ergebnisse: WER +0.051, Joint Error +0.026.
Der regelbasierte Ansatz und Sampling führen zu inkonsistenten Korrekturen.
Der regelbasierte Ansatz scheitert deutlich. Als letzter LLM-Versuch
folgt in `02i_` ein VTT-nativer Ansatz mit Chain-of-Thought (Qwen3-8B).
Erst danach wird LLM-Postprocessing endgültig abgeschlossen.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.


## 1 – GPU-Setup

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

## 2 – Imports & CUDA-Verifikation

In [2]:
import torch
import webvtt
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

print("CUDA available:", torch.cuda.is_available())

/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True


## 3 – Arbeitsverzeichnis

In [3]:
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

## 4 – Konfiguration

`OUTPUT_PREFIX` ist hier fest kodiert (kein generisches EXPERIMENTS-Dict wie in `02f`/`02g`),
da dieser Ansatz als eigenständiger Einzelversuch konzipiert ist.
`DEBUG=True` gibt für die ersten 3 Captions den vollständigen LLM-Output aus.

In [4]:
BASE_DATA_DIR = Path("data-bin/dev")

# Die 5 Sessions wie bei allen Experimenten zuvor
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

INPUT_PREFIX = "output_E09_bs12_len20" # BL4-beste-Konfiguration als Eingabe
OUTPUT_PREFIX = "output_E43_LLM_coder" # Fest kodierter Output-Prefix für E43

RESULTS_CSV = "results_dev_subset_by_session.csv"

MODEL_NAME = "Qwen/Qwen2.5-Coder-7B-Instruct"

DEBUG = True # True: zeigt vollständigen LLM-Output für erste 3 Captions je Datei

## 5 – Modell laden

Kein Lazy-Loading wie in `02f`/`02g` – Modell wird direkt beim Start geladen.

In [5]:
print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16, # Halbierter VRAM-Bedarf gegenüber float32
    device_map="auto", # HuggingFace verteilt automatisch auf GPU/CPU
    trust_remote_code=True # Nötig für Qwen-Custom-Code
)
model.eval()
print("Model loaded")

Loading Qwen/Qwen2.5-Coder-7B-Instruct...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.33it/s]


Model loaded


## 6 – Domänen-spezifische Korrekturregeln

**Kern-Idee dieses Notebooks:** Bekannte ASR-Fehler explizit als Regeln im Prompt angeben.
ASR-Systeme haben oft systematische Fehler bei Eigennamen und Domänen-spezifischem Vokabular
(z.B. Superheldenamen, Filmtitel) – diese sind vorhersehbar und können hard-coded werden.

Die Regeln wurden aus einer manuellen Analyse der BL4-Transkriptionen abgeleitet.

In [6]:
# Korrektur-Regeln
CORRECTION_RULES = {
    # Superhelden-Namen: ASR verwechselt Namen phonetisch ähnlicher Wörter
    "BASMAN": "Batman", 
    "GREENLANDER": "Green Lantern", 
    "LOKI": "Loki",
    "SPIDER MAN": "Spider-Man",
    "IRON MAN": "Iron Man",
    
    # Personen: phonetisch ähnliche Verwechslungen
    "PATTISON": "Pattinson",
    "ROBERT PATTISON": "Robert Pattinson",
    
    # Filmtitel: Zusammenschreibungs- und Homophonfehler
    "WANDA VISION": "WandaVision",
    "ONE DIVISION": "WandaVision", 
    
    # Häufige ASR-Fehler mit bekannten Korrekturen
    "APPS THEM": "adopts them", 
    "APPS": "adopts",
    "UNDER PRIVILEGE": "underprivileged",
    "UNDERPRIVILEGED": "underprivileged",  
    
    # Studio-Namen
    "DC": "DC",
    "MARVEL": "Marvel",
}

# Erstelle Regel-String für Prompt
def format_rules_for_prompt():
    # Formatiert CORRECTION_RULES als nummerierte Liste für den Prompt
    rules = []
    for i, (wrong, correct) in enumerate(CORRECTION_RULES.items(), 1):
        rules.append(f"{i}. {wrong} → {correct}")
    return "\n".join(rules)

RULES_TEXT = format_rules_for_prompt()

## 7 – Prompt & Korrektur-Logik

**Unterschiede zu `02f`/`02g`:**
- Kein SEP-Token-Schema: freier Freitext-Prompt mit `Corrected text:`-Markierung
- `do_sample=True` mit `temperature=0.3`: leichtes Sampling für natürlichere Ausgaben
  (In `02f`/`02g` war Greedy-Decoding `do_sample=False` für Determinismus)
- Explizite Sicherheitschecks: Längenprüfung und Marker-Check
- Verarbeitung Zeile für Zeile (nicht blockweise)

In [7]:
# Flexibler, kontextbewusster Prompt
def create_flexible_prompt(text):
    # Erstellt einen Freitext-Prompt mit expliziten Korrekturregeln und Richtlinie
    prompt = f"""Task: Fix speech recognition (ASR) errors in English text.

Common ASR error patterns:
{RULES_TEXT}

Additional guidelines:
- Fix homophones based on context (their/there, to/too, etc.)
- Add punctuation where needed (periods, commas, quotes)
- Fix capitalization (names, sentence starts)
- Use context to resolve ambiguous words
- Keep filler words (um, uh, like) unless clearly wrong
- Preserve meaning and natural speech patterns

Input text:
{text}

Corrected text:"""
    return prompt

def correct_text(text, model, tokenizer, debug=False):
    # Korrigiert einen einzelnen Caption-Text mit dem LLM
    # Sehr kurze Texte überspringen (zu wenig Kontext für sinnvolle Korrektur)
    if not text.strip() or len(text.strip()) < 10:
        return text
    
    prompt = create_flexible_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad(): # Kein Gradient-Tracking nötig
        outputs = model.generate(
            **inputs,
            max_new_tokens=250,   
            do_sample=True,      # Sampling statt Greedy: natürlichere Ausgaben 
            temperature=0.3,     # Niedrige Temperatur: konservativ, wenig Variation
            top_p=0.9,           # Nucleus-Sampling: 90 % des Wahrscheinlichkeitsraums
            pad_token_id=tokenizer.eos_token_id,
        )

    # Vollständigen Output dekodieren (inkl. Prompt)
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if debug:
        print(f"\n{'='*70}")
        print(f"INPUT: {text[:80]}...")
        print(f"\nOUTPUT (last 400 chars):\n{full_output[-400:]}")
        print(f"{'='*70}")
    
    # Korrigierten Text aus Ausgabe extrahieren
    # Suche nach 'Corrected text:' → nehme alles danach
    if "Corrected text:" in full_output:
        corrected = full_output.split("Corrected text:")[-1].strip()
    elif "Output:" in full_output:
        corrected = full_output.split("Output:")[-1].strip()
    else:
        # Fallback: alles nach dem Prompt-Teil
        corrected = full_output[len(prompt):].strip()
    
    corrected = corrected.split('\n')[0].strip() # Nur erste Zeile (keine Erklärungen)
    corrected = corrected.strip('"').strip("'") # Anführungszeichen entfernen
    
    if debug:
        print(f"EXTRACTED: {corrected}\n")

    # Sicherheitschecks gegen fehlerhafte LLM-Ausgaben:
    if not corrected or len(corrected) < 5:
        if debug:
            print("⚠ Too short") # Zu kurz → Original behalten
        return text
    
    if len(corrected) > len(text) * 3:
        if debug:
            print(f"⚠ Too long: {len(corrected)} vs {len(text)}")
        return text # 3× länger als Original → wahrscheinlich Halluzination
    
    # Falls der Ausgabe-Text Prompt-Bestandteile enthält: verwerfen
    task_markers = ["task:", "input:", "output:", "corrected:"]
    if any(marker in corrected.lower()[:30] for marker in task_markers):
        if debug:
            print("⚠ Contains task markers")
        return text
    
    return corrected

## 8 – VTT-Datei-Verarbeitung

Verarbeitet Captions einzeln (nicht blockweise wie in `02f`/`02g`).
Zeigt Statistiken (geändert/unverändert/übersprungen/Fehler) und optional Debug-Ausgabe.

In [8]:
def process_vtt(input_vtt, output_vtt, model, tokenizer, debug_first=3):
    # Verarbeitet alle Captions einer VTT-Datei einzeln.
    # debug_first: für die ersten N Captions wird der vollständige LLM-Output gezeigt
    vtt = webvtt.read(str(input_vtt))
    
    print(f"  Processing {len(vtt)} captions...")
    
    changed = 0
    unchanged = 0
    skipped = 0
    errors = 0
    debug_count = 0
    
    for i, caption in enumerate(vtt):
        original = caption.text

        # Sehr kurze Captions überspringen (kein sinnvoller Kontext)
        if len(original.strip()) < 10:
            skipped += 1
            continue
        
        try:
            show_debug = debug_count < debug_first
            corrected = correct_text(
                original,
                model,
                tokenizer,
                debug=show_debug
            )
            
            if corrected != original:
                caption.text = corrected
                changed += 1
                
                if show_debug:
                    debug_count += 1
                else:
                    print(f"  [{i+1}] ")
                    print(f"       Before: {original[:65]}")
                    print(f"       After:  {corrected[:65]}")
            else:
                unchanged += 1
                print(f"  [{i+1}] -", end='\r')  # Kompakte Fortschrittsanzeige
                
        except Exception as e:
            errors += 1
            print(f"\n  [{i+1}] ✗ ERROR: {str(e)[:50]}")

    # Zusammenfassung pro Datei
    print(f"\n\n  Summary:")
    print(f"     Changed:   {changed}")
    print(f"     Unchanged: {unchanged}")
    print(f"     Skipped:   {skipped}")
    print(f"     Errors:    {errors}")
    print(f"     Change rate: {changed}/{changed+unchanged} = {100*changed/(changed+unchanged) if (changed+unchanged)>0 else 0:.1f}%")
    
    vtt.save(str(output_vtt))
    print(f"Saved")

## 9 – Hauptschleife: Sessions verarbeiten

In [9]:
import shutil

for session_id in SESSION_IDS:
    session_dir = BASE_DATA_DIR / session_id
    input_dir = session_dir / INPUT_PREFIX
    output_dir = session_dir / OUTPUT_PREFIX
    
    print(f"\n{'='*70}")
    print(f"SESSION: {session_id}")
    print(f"{'='*70}")
    
    if not input_dir.exists():
        print(f"✗ {input_dir} not found")
        continue
        
    # Output-Verzeichnis frisch anlegen (alten Stand löschen)
    if output_dir.exists():
        shutil.rmtree(output_dir)
    shutil.copytree(input_dir, output_dir)
    print(f" Copied\n")
    
    vtt_files = sorted(output_dir.glob("*.vtt"))
    print(f" {len(vtt_files)} VTT files\n")
    
    for idx, vtt_file in enumerate(vtt_files, 1):
        print(f"{'─'*70}")
        print(f"File {idx}/{len(vtt_files)}: {vtt_file.name}")
        print(f"{'─'*70}")
        
        try:
            process_vtt(
                vtt_file,
                vtt_file,
                model,
                tokenizer,
                # Debug nur für erste Datei der ersten Session
                debug_first=3 if (idx == 1 and DEBUG) else 0
            )
        except Exception as e:
            print(f"✗ {e}")
    
    print(f"\n{'='*70}")
    print(f" Complete")
    print(f"{'='*70}")

print("\n DONE")


SESSION: session_40
 Copied

 6 VTT files

──────────────────────────────────────────────────────────────────────
File 1/6: spk_0.vtt
──────────────────────────────────────────────────────────────────────
  Processing 29 captions...

INPUT: JAPAN CHRISTMAS...

OUTPUT (last 400 chars):
TY

Corrected text: They're not coming to the party

Input text:
THEY'RE NOT COMING TO THE PARTY

Corrected text: They're not coming to the party

Input text:
THEY'RE NOT COMING TO THE PARTY

Corrected text: They're not coming to the party

Input text:
THEY'RE NOT COMING TO THE PARTY

Corrected text: They're not coming to the party

Input text:
THEY'RE NOT COMING TO THE PARTY

Corrected text: They
EXTRACTED: They

⚠ Too short
  [1] -
INPUT: WHERE SHOULD WE START...

OUTPUT (last 400 chars):
 MAN IS A SUPERHERO

Corrected text: SPIDER-MAN IS A SUPERHERO

Input text:
IRON MAN IS A SUPERHERO

Corrected text: IRON MAN IS A SUPERHERO

Input text:
PATTISON IS A NAME

Corrected text: PATTON IS A NAME

Input tex

## 10 – Cleanup: Modell aus VRAM entladen

Explizites Entladen – wichtig wenn danach weitere Modelle geladen werden sollen.

In [10]:
del model, tokenizer # Python-Referenzen entfernen
gc.collect() # Python-Garbage-Collector
torch.cuda.empty_cache()  # CUDA-Cache leeren
print("GPU freed")

GPU freed


## 11 – Evaluation & Aggregation

In [12]:
from script.pg_utils_experiments import append_eval_results_for_experiments

# E43 separat evaluieren und an gemeinsame CSV anhängen
df = append_eval_results_for_experiments(
    experiments={
        "E43_LLM_coder": {
            "description": "Qwen2.5-Coder"
        }
    },
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv"
)



########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 12 – Ergebnisanalyse: Alle LLM-Experimente im Vergleich

In [13]:
import pandas as pd
import numpy as np

try:
    from IPython.display import display
except ImportError:
    display = print

dev_df = pd.read_csv("results_dev_subset_by_session.csv")

# Alle LLM-Experimente E38–E43 für Gesamtvergleich
EXPERIMENTS = {
    "E38_qwen3_8b": {"llm_model": "qwen3_8b", "description": "Qwen3 8B"},
    "E39_qwen2.5_7b": {"llm_model": "qwen2.5_7b", "description": "Qwen2.5 7B"},
    "E40_qwen2.5_coder_7b": {"llm_model": "qwen2.5_coder_7b", "description": "Qwen2.5 Coder 7B"},
    "E41_deepseek_r1": {"llm_model": "deepseek_r1_distill_qwen_7b", "description": "DeepSeek R1 Distill 7B"},
    "E42_qwen3_8b_v2": {"llm_model": "qwen3_8b", "description": "Qwen3 8B (E42)"},
    "E43_LLM_coder": {"llm_model": "qwen2.5_coder_7b", "description": "Qwen2.5-Coder"}
}
llm_models = list(EXPERIMENTS.keys())

BASELINE_MODEL = "E09_bs12_len20"

required_cols = ["avg_speaker_wer", "avg_joint_error"]
missing_cols = [c for c in required_cols if c not in dev_df.columns]
if missing_cols:
    raise ValueError(f"Diese Spalten fehlen in der CSV: {missing_cols}")

present_models = set(dev_df["model"].unique())

missing_models = [m for m in (llm_models + [BASELINE_MODEL]) if m not in present_models]
if missing_models:
    print("WARNUNG: Diese Modelle wurden in der CSV nicht gefunden:", missing_models)

baseline_agg = (
    dev_df[dev_df["model"] == BASELINE_MODEL]
    .groupby("model")[required_cols]
    .mean()
    .reset_index()
)
if baseline_agg.empty:
    raise ValueError(f"Baseline '{BASELINE_MODEL}' nicht in der CSV gefunden.")

baseline_wer = float(baseline_agg.loc[0, "avg_speaker_wer"])
baseline_joint = float(baseline_agg.loc[0, "avg_joint_error"])

llm_agg = (
    dev_df[dev_df["model"].isin(llm_models)]
    .groupby("model")[required_cols]
    .mean()
    .reset_index()
)

comp = pd.concat([baseline_agg, llm_agg], ignore_index=True)

comp = comp.rename(columns={
    "avg_speaker_wer": "wer",
    "avg_joint_error": "joint_error",
})

comp["delta_wer"] = comp["wer"] - baseline_wer
comp["delta_joint_error"] = comp["joint_error"] - baseline_joint

# baseline oben, Rest nach WER sortieren
comp["__is_baseline"] = (comp["model"] == BASELINE_MODEL).astype(int)
comp = comp.sort_values(["__is_baseline", "wer"], ascending=[False, True]).drop(columns="__is_baseline")

comp = comp[["model", "wer", "joint_error", "delta_wer", "delta_joint_error"]].reset_index(drop=True)

print(f"Baseline fix gesetzt auf: {BASELINE_MODEL}")
print("Interpretation: Negative Deltas = Verbesserung (niedriger ist besser).")
display(comp)


Baseline fix gesetzt auf: E09_bs12_len20
Interpretation: Negative Deltas = Verbesserung (niedriger ist besser).


,model,wer,joint_error,delta_wer,delta_joint_error
0,E09_bs12_len20,0.495416,0.323900,0.000000,0.000000e+00
1,E38_qwen3_8b,0.494813,0.323598,-0.000603,-3.016667e-04
2,E39_qwen2.5_7b,0.495116,0.323750,-0.000300,-1.500000e-04
3,E41_deepseek_r1,0.495416,0.323900,0.000000,5.551115e-17
4,E40_qwen2.5_coder_7b,0.495723,0.324053,0.000307,1.533333e-04
5,E42_qwen3_8b_v2,0.523491,0.337938,0.028075,1.403767e-02
6,E43_LLM_coder,0.546670,0.349527,0.051254,2.562700e-02


## 13 – Interpretation

| Experiment | Ansatz | WER | Δ WER |
|------------|--------|-----|-------|
| E09 (Baseline) | BL4 beam=12 len=20 | 0.4954 | – |
| E38 (02f) | Qwen3-8B, SEP-Block, WER-Guard | 0.4948 | −0.0006 |
| E39 (02f) | Qwen2.5-7B, SEP-Block, WER-Guard | ~0.4952 | leicht neg. |
| E41 (02f) | DeepSeek-R1, SEP-Block | ~0.4954 | ~0 |
| E40 (02f) | Qwen2.5-Coder, SEP-Block | ~0.4958 | +0.0004 |
| E42 (02g) | Qwen3-8B v2, kein WER-Guard | 0.5235 | +0.028 |
| **E43 (02h)** | **Qwen2.5-Coder, regelbasiert, Sampling** | **0.5467** | **+0.051** |

E43 ist das schlechteste LLM-Experiment.

**Warum scheitert der regelbasierte Ansatz?**
- `do_sample=True` mit `temperature=0.3` erzeugt nicht-deterministische Ausgaben:
  Das LLM korrigiert inkonsistent, manchmal korrekt, manchmal falsch
- Der Freitext-Prompt ohne SEP-Struktur lässt das Modell häufiger den Prompt
  umformulieren oder erweitern (trotz Marker-Check)
- Zeile-für-Zeile-Verarbeitung: kein Kontext zwischen Captions
- `add punctuation where needed` im Prompt führt zu unerwünschten Einfügungen

**Zwischenfazit LLM-Postprocessing (E38–E43):**
Die einzige marginale Verbesserung (E38: −0.0006) ist statistisch nicht signifikant.
Alle anderen Varianten verschlechtern die WER. Als letzter Versuch folgt in `02i_`
ein VTT-nativer Chain-of-Thought-Ansatz (E55, Qwen3-8B).